In [1]:
import os
import cv2
import mediapipe as mp
import pprint
import numpy as np
import pandas as pd
import torch

robienie dataset

In [ ]:

# merging data
from pathlib import Path
import shutil

dataset_dir = "Sign-Language-Digits-Dataset-master\Dataset"
dataset_dir_2 = "data_2"

dataset_dirs = [dataset_dir , dataset_dir_2]

out_dir = Path("merged_dataset")
out_dir.mkdir(exist_ok=True)

for d in dataset_dirs:
    for cls in os.listdir(d):
        src = Path(d) / cls
        if not src.is_dir(): continue
        dst = out_dir / cls
        dst.mkdir(exist_ok=True)
        for f in os.listdir(src):
            shutil.copy(src / f, dst / f)




In [5]:

mp_hands =  mp.solutions.hands
hands_model  = mp_hands.Hands(static_image_mode=True, max_num_hands=1 , min_detection_confidence=0.1)


data = []
errors = []

count = 0
merged_data_dir = "merged_dataset"
for class_name in os.listdir(merged_data_dir):
    class_dir = os.path.join(merged_data_dir , class_name)


    for image_file in os.listdir(class_dir):
        count +=1
        image_path = os.path.join(class_dir , image_file)
        image  =cv2.imread(image_path)
        image =  cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image , (128 , 128))
        results = hands_model.process(image)
        
        if results.multi_hand_landmarks:
            hand_landmarks = results.multi_hand_landmarks[0]
            points = torch.tensor([[p.x , p.y , p.z] for p in hand_landmarks.landmark])
            data.append((points ,class_name))
        else:
            errors.append(image_path)
        
        if count%1000 == 0:
            print(count , len(errors))

   
torch.save(data , "tensordata" )        


1000 21
2000 52
3000 95
4000 98
5000 102
6000 111
7000 111
8000 111
9000 137
10000 145
11000 152
12000 166
13000 166
14000 166
15000 166
16000 166
17000 166
18000 166
19000 166
20000 166
21000 166
22000 166
23000 166
24000 166
25000 166
26000 166
27000 166
28000 166
29000 166
30000 168
31000 187
32000 432
33000 524
34000 534
35000 552
36000 577
37000 577
38000 578
39000 578
40000 578
41000 578
42000 578
43000 578
44000 578


In [ ]:
image_path = "merged_dataset/S/0.JPG"
image = cv2.imread(image_path)
print("Shape obrazu:", image.shape)  

Shape obrazu: (128, 128, 3)


In [ ]:
#moj komputer nie daje rady z przetwarzaniem większej ilości niż 

from concurrent.futures import ThreadPoolExecutor


merged_data_dir = "merged_dataset"


def process_image(class_name, img_file):

    mp_hands = mp.solutions.hands
    hands_model = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.1)    
    
    img_path = os.path.join(merged_data_dir, class_name, img_file)
    
    image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    
    results = hands_model.process(image)
    
    if results.multi_hand_landmarks:
        
        pts = torch.tensor([[p.x, p.y, p.z] for p in results.multi_hand_landmarks[0].landmark])
        
        return (pts, int(class_name))
    return None


tasks = [(class_name, img_file) 
         
         for class_name in os.listdir(merged_data_dir)
         
         for img_file in os.listdir(os.path.join(merged_data_dir, class_name))]

data = []

count = 0
with ThreadPoolExecutor(max_workers=2) as executor:
    
    for res in executor.map(lambda t: process_image(*t), tasks):
        count +=1
        if res is not None:
            
            data.append(res)
        if count % 1000 ==0:
            print(f"{count} ilość obrazów {len(data)} ")

torch.save(data, "hands_tensor_data_multithread.pt")
print(f"Gotowe! Liczba próbek: {len(data)}")

: 

In [ ]:
total = sum(len(files) for _, _, files in os.walk(merged_data_dir))
print("Liczba wszystkich plików:", total)

Liczba wszystkich plików: 44062


: 

szacowanie ile zajmie przejście przez dataset

In [ ]:
import time
mp_hands = mp.solutions.hands
hands_model = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.1)

image_path = r"merged_dataset\0\IMG_1128.JPG"  

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
start = time.time()

results = hands_model.process(image)

if results.multi_hand_landmarks:
    hand_landmarks = results.multi_hand_landmarks[0]
    points = torch.tensor([[p.x, p.y, p.z] for p in hand_landmarks.landmark])
    print("Tensor shape:", points.shape)

end = time.time()
print("Czas przetwarzania:", end - start, "sekund")

Tensor shape: torch.Size([21, 3])
Czas przetwarzania: 0.06308627128601074 sekund


In [3]:
os.cpu_count()

6

In [2]:
tensor_data =  torch.load("tensordata")

In [3]:
labels = os.listdir("merged_dataset")
label_map = {k : v  for v , k in enumerate(labels)}

X = torch.stack([t[0] for t in tensor_data])
Y = torch.stack([torch.tensor(label_map[t[1]]) for t in tensor_data])
X.shape



torch.Size([43484, 21, 3])

In [4]:
X.shape

torch.Size([43484, 21, 3])

In [5]:
from torch.utils.data import TensorDataset ,DataLoader ,random_split


dataset = TensorDataset(X , Y)

train_len = int(0.8 * len(dataset))
valid_len= len(dataset) - train_len

train_dataset , valid_dataset = random_split(dataset , [train_len , valid_len])

traning_loader = DataLoader( train_dataset , batch_size= 32, shuffle=True)
valid_loader = DataLoader(valid_dataset , batch_size=32 , shuffle=False)


In [6]:
label_map

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 'A': 10,
 'B': 11,
 'C': 12,
 'D': 13,
 'E': 14,
 'F': 15,
 'G': 16,
 'H': 17,
 'I': 18,
 'J': 19,
 'K': 20,
 'L': 21,
 'M': 22,
 'N': 23,
 'O': 24,
 'P': 25,
 'Q': 26,
 'R': 27,
 'S': 28,
 'T': 29,
 'U': 30,
 'V': 31,
 'W': 32,
 'X': 33,
 'Y': 34,
 'Z': 35}

In [16]:
out_size =len(label_map)

In [17]:
import torch.nn as nn
from torch.nn import Module  

class Feather_MLP(Module):
    def __init__(self , xyz , hidden_size ,feathers_size ):
        super().__init__()
        self.fc1 = nn.Linear(xyz , hidden_size , bias=False)
        self.bn1 = nn.BatchNorm1d(hidden_size)

        self.fc2 = nn.Linear(hidden_size , hidden_size , bias=False)
        self.bn2 = nn.BatchNorm1d(hidden_size)

        self.out = nn.Linear(hidden_size , feathers_size)

        self.leakRL = nn.LeakyReLU(0.1)

    def forward(self, x ):
        B , N , _ = x.shape

        #1 layer
        x = self.fc1(x)
        x = x.transpose(1,2) # (B,N,C) --> (B,C,N) for BatchNorm
        x = self.bn1(x).transpose(1,2)
        x = self.leakRL(x)

        #2 layer
        x = self.fc2(x)
        x=self.bn2(x.transpose(1,2)).transpose(1,2)
        x = self.leakRL(x)

        #out
        x = self.out(x)
        x= self.leakRL(x)

        return x # (B,  N , feather_size)
    
class Sign_language_PointNet(Module):
    def __init__(self , mlp , hidden_size, out_size ):
        super().__init__()
        self.feather_mlp = mlp
        feather_size = mlp.out.out_features
        
        self.fc1 = nn.Linear(feather_size ,hidden_size )
        self.bn1 = nn.BatchNorm1d(hidden_size)

        self.fc2 = nn.Linear(hidden_size , hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        
        self.out = nn.Linear(hidden_size ,out_size)

        self.leakRL = nn.LeakyReLU(0.1)    

    
    def forward(self,x):
        x = self.feather_mlp(x)

        x = torch.max(x , dim=1)[0] #(B,feathers)
        #layer  1 
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.leakRL(x)

        #layer 2 
        x = self.fc2(x)
        x =self.bn2(x)
        x = self.leakRL(x)
        

        x = self.out(x)

        return x 

    def detect(self):
        pass
        

test 

In [20]:
from math import log

test_run_mlp = Feather_MLP(3,64,64)
test_run_model = Sign_language_PointNet(test_run_mlp , 128,out_size)

c = nn.CrossEntropyLoss()
for x , y in traning_loader:
    
    output= test_run_model(x)
    loss = c(output,y)
    print(loss.item())
    break

print(log(out_size))

3.6904783248901367
3.58351893845611


In [21]:
import optuna

def train_eval(model,lr, train_data, valid_data , epoch = 5, device = "cuda"):
    
    optimizer = torch.optim.Adam(model.parameters() , lr)
    criterion = nn.CrossEntropyLoss()


    train_losses, valid_losses = [] , []
    
    
    for _ in range(epoch):
        model.to(device)
        model.train()
        
        running_loss = 0.0
        for x , y in train_data:
            optimizer.zero_grad()
            x , y = x.to(device) , y.to(device)
            output = model(x)
            loss = criterion(output , y)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * y.size(0)
        train_loss = running_loss/ len(train_data.dataset)
        train_losses.append(train_loss)

        model.eval()
        with torch.no_grad():
            running_loss = 0.0
        for x , y in valid_data:
            x , y = x.to(device) , y.to(device)
            output = model(x)
            loss = criterion(output , y)
            
            running_loss += loss.item() * y.size(0)
        valid_loss = running_loss/ len(valid_data.dataset)
        valid_losses.append(valid_loss)
    
    return train_losses , valid_losses




def objective(trail):

    mlp_hidden_size = trail.suggest_int("mlp_hidden_size" ,32,512)
    feather_size = trail.suggest_int("feather_size" ,32,512)
    hidden_size = trail.suggest_int("hidden_size" , 32, 512)
    lr = trail.suggest_float("lr" , 1e-5, 1e-3 , log=True)  


    mlp = Feather_MLP(xyz =3,hidden_size= mlp_hidden_size , feathers_size= feather_size )
    model = Sign_language_PointNet(mlp = mlp , hidden_size= hidden_size, out_size = out_size)
    _ , valid_loss = train_eval(model , lr , traning_loader , valid_loader)
    

    return valid_loss[-1]

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=200)

[I 2025-10-22 18:32:30,833] A new study created in memory with name: no-name-78182fd7-43b6-4935-b6a3-50eb4aadc337
[I 2025-10-22 18:33:00,055] Trial 0 finished with value: 0.024612917577017068 and parameters: {'mlp_hidden_size': 503, 'feather_size': 78, 'hidden_size': 490, 'lr': 3.0244153174667385e-05}. Best is trial 0 with value: 0.024612917577017068.
[I 2025-10-22 18:33:26,349] Trial 1 finished with value: 0.07924381523169607 and parameters: {'mlp_hidden_size': 287, 'feather_size': 228, 'hidden_size': 273, 'lr': 1.2472024603336184e-05}. Best is trial 0 with value: 0.024612917577017068.
[I 2025-10-22 18:33:50,900] Trial 2 finished with value: 0.010363463603231678 and parameters: {'mlp_hidden_size': 289, 'feather_size': 444, 'hidden_size': 211, 'lr': 0.00023980995899293615}. Best is trial 2 with value: 0.010363463603231678.
[I 2025-10-22 18:34:13,712] Trial 3 finished with value: 0.023469094986735755 and parameters: {'mlp_hidden_size': 344, 'feather_size': 215, 'hidden_size': 118, 'lr':

In [23]:
study.best_params

{'mlp_hidden_size': 213,
 'feather_size': 448,
 'hidden_size': 356,
 'lr': 0.00011792606036961329}

In [24]:
sum(p.numel() for p in test_run_model.parameters())

38692

In [25]:
p = study.best_params
hidden_size_mlp = p.get("mlp_hidden_size")
feather_size = p.get("feather_size")
hidden_size = p.get("hidden_size")
lr = p.get("lr")

feather_model = Feather_MLP(xyz=3 , hidden_size=hidden_size_mlp, feathers_size=feather_size )
model = Sign_language_PointNet(mlp = feather_model , hidden_size=hidden_size ,out_size=out_size )

train_eval(model,lr,traning_loader , valid_loader)

([0.29494858348030684,
  0.03781278924879717,
  0.023460163090367445,
  0.01906933470631806,
  0.014903049271328976],
 [0.04393798164120382,
  0.029433356711316055,
  0.018443971505883137,
  0.015622548763909545,
  0.012485544218678432])

In [28]:
torch.save(model.state_dict() , "model.pt")